# MEST DAY 6

## Morning Session
### Transfer Learning for Computer Vision

In [ ]:
!pip install tensorflow-gpu==2.0.0-beta0

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import pandas as pd
import numpy as np

print(tf.__version__)

In [ ]:
LIST_OF_LABELS = "cirrus,cumulonimbus,cumulus".split(',')
HEIGHT = 300
WIDTH = 300
NUM_CHANNELS = 3
NCLASSES = len(LIST_OF_LABELS)
BATCH_SIZE = 32

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### Get a list of files and their classes

In [ ]:
_path = '/content/gdrive/My Drive/clouds/'
_files = list()
_classes = list()

for i in range(1,21):
  _files.append('{}cirrus/{}.jpg'.format(_path,i))
  _classes.append(0)
  _files.append('{}cumulonimbus/{}.jpg'.format(_path,i))
  _classes.append(1)
  _files.append('{}cumulus/{}.jpg'.format(_path,i))
  _classes.append(2)

### Load and preprocess images

In [ ]:
def preprocess_image(image):
  image = tf.image.decode_image(contents=image, channels=NUM_CHANNELS, expand_animations=False)
  image = tf.image.resize(image, [HEIGHT, WIDTH])
  image /= 255.0  # normalize to [0,1] range

  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [ ]:
filename = _files[1]
img = tf.io.read_file(filename)
#img = tf.image.decode_image(contents=img, channels=NUM_CHANNELS, expand_animations=False)
img = preprocess_image(img)
print(img)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(img)

### Image Preprocessing

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(_files)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
image_ds

In [ ]:
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(_classes, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
image_label_ds

In [ ]:
ds = image_label_ds.shuffle(buffer_size=1000 * BATCH_SIZE)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

### LeNet-5

In [ ]:
model = models.Sequential([
    layers.Conv2D(filters=6, kernel_size=(5,5), strides=1, activation='tanh', input_shape=(HEIGHT, HEIGHT, NUM_CHANNELS)),
    layers.AveragePooling2D(pool_size=(2,2)),
    layers.Conv2D(filters=16, kernel_size=(5,5), strides=1, activation='tanh'),
    layers.AveragePooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(120),
    layers.Dense(84),
    layers.Dense(NCLASSES)
])
print(model.summary())

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9, decay=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(ds, epochs=5, steps_per_epoch=60)

### VGG Architecture

In [ ]:
model = models.Sequential([
    layers.Conv2D(filters=96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), bias_initializer=tf.initializers.zeros(), kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=256, kernel_size=(5, 5), activation='relu', bias_initializer=tf.initializers.ones(), kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=384, kernel_size=(3, 3), activation='relu', bias_initializer=tf.initializers.zeros(), kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.Conv2D(filters=384, kernel_size=(3, 3), activation='relu', bias_initializer=tf.initializers.ones(), kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', bias_initializer=tf.initializers.ones(), kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(4096, activation='relu', bias_initializer=tf.initializers.ones(), kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu', bias_initializer=tf.initializers.ones(), kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu', kernel_initializer=tf.initializers.RandomNormal(mean=0, stddev=0.01)),
    layers.Dense(NCLASSES, activation='softmax', bias_initializer=tf.initializers.ones())
])

print(model.summary())

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(ds, epochs=5, steps_per_epoch=60)

### VGG Pre-trained

In [ ]:
base_model = tf.keras.applications.vgg19.VGG19(input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), include_top=False, weights='imagenet')
base_model.trainable = False
print(base_model.summary())

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NCLASSES, activation='softmax')
])
print(model.summary())

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(ds, epochs=5, steps_per_epoch=60)

### Functional API

In [ ]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(3, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=x)
print(model.summary())

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(ds, epochs=10, steps_per_epoch=60)

## Afternoon Session
### Cloud ML Engine
### Hyperparameter Tuning
### Distributed Training